In [1]:
import pandas as pd
import numpy as np
import os
from IPython.display import display
pd.set_option('display.max_columns', None)
import geopandas as gpd
from shapely import Point, Polygon, MultiPolygon
import contextily as ctx

import mapclassify
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from scipy.spatial import Voronoi
import osmnx as ox
ox.config(log_console=True, use_cache=True)
import pyproj
import math
from tqdm.notebook import tqdm
tqdm.pandas()
from collections import defaultdict
import osmnx as ox
import os

In [2]:
PATH_PROJECT = '..'
PATH_DATA = f'{PATH_PROJECT}/data'

выгружаем данные из OSM

In [3]:
https://download.geofabrik.de/europe/belarus-latest-free.shp.zip
https://download.geofabrik.de/asia/armenia-latest-free.shp.zip
https://download.geofabrik.de/asia/kazakhstan-latest-free.shp.zip
https://download.geofabrik.de/russia/central-fed-district-latest-free.shp.zip
https://download.geofabrik.de/russia/crimean-fed-district-latest-free.shp.zip
https://download.geofabrik.de/russia/far-eastern-fed-district-latest-free.shp.zip
https://download.geofabrik.de/russia/north-caucasus-fed-district-latest-free.shp.zip
https://download.geofabrik.de/russia/northwestern-fed-district-latest-free.shp.zip
https://download.geofabrik.de/russia/siberian-fed-district-latest-free.shp.zip
https://download.geofabrik.de/russia/south-fed-district-latest-free.shp.zip
https://download.geofabrik.de/russia/ural-fed-district-latest-free.shp.zip
https://download.geofabrik.de/russia/volga-fed-district-latest-free.shp.zip
https://download.geofabrik.de/russia/kaliningrad-latest-free.shp.zip

In [18]:
! wget -P {PATH_DATA+'/processed'} https://download.geofabrik.de/europe/belarus-latest-free.shp.zip
! wget -P {PATH_DATA+'/processed'} https://download.geofabrik.de/asia/armenia-latest-free.shp.zip
! wget -P {PATH_DATA+'/processed'} https://download.geofabrik.de/asia/kazakhstan-latest-free.shp.zip
! wget -P {PATH_DATA+'/processed'} https://download.geofabrik.de/russia/central-fed-district-latest-free.shp.zip
! wget -P {PATH_DATA+'/processed'} https://download.geofabrik.de/russia/crimean-fed-district-latest-free.shp.zip
! wget -P {PATH_DATA+'/processed'} https://download.geofabrik.de/russia/far-eastern-fed-district-latest-free.shp.zip
! wget -P {PATH_DATA+'/processed'} https://download.geofabrik.de/russia/north-caucasus-fed-district-latest-free.shp.zip
! wget -P {PATH_DATA+'/processed'} https://download.geofabrik.de/russia/northwestern-fed-district-latest-free.shp.zip
! wget -P {PATH_DATA+'/processed'} https://download.geofabrik.de/russia/siberian-fed-district-latest-free.shp.zip
! wget -P {PATH_DATA+'/processed'} https://download.geofabrik.de/russia/south-fed-district-latest-free.shp.zip
! wget -P {PATH_DATA+'/processed'} https://download.geofabrik.de/russia/ural-fed-district-latest-free.shp.zip
! wget -P {PATH_DATA+'/processed'} https://download.geofabrik.de/russia/volga-fed-district-latest-free.shp.zip
! wget -P {PATH_DATA+'/processed'} https://download.geofabrik.de/russia/kaliningrad-latest-free.shp.zip

--2023-12-10 14:21:06--  https://download.geofabrik.de/europe/belarus-latest-free.shp.zip
Resolving download.geofabrik.de (download.geofabrik.de)... 65.109.48.72, 65.109.50.43
Connecting to download.geofabrik.de (download.geofabrik.de)|65.109.48.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 552062892 (526M) [application/zip]
Saving to: ‘../data/processed/belarus-latest-free.shp.zip.1’

                 be  12%[=>                  ]  64,28M  7,86MB/s    eta 59s    ^C


In [17]:
!unzip {PATH_DATA+'/processed/belarus-latest-free.shp.zip'} -d {PATH_DATA+'/processed/belarus-latest-free.shp'}
!unzip {PATH_DATA+'/processed/armenia-latest-free.shp.zip'} -d {PATH_DATA+'/processed/armenia-latest-free.shp'}
!unzip {PATH_DATA+'/processed/kazakhstan-latest-free.shp.zip'} -d {PATH_DATA+'/processed/kazakhstan-latest-free.shp'}
!unzip {PATH_DATA+'/processed/central-fed-district-latest-free.shp.zip'} -d {PATH_DATA+'/processed/central-fed-district-latest-free.shp'}
!unzip {PATH_DATA+'/processed/crimean-fed-district-latest-free.shp.zip'} -d {PATH_DATA+'/processed/crimean-fed-district-latest-free.shp'}
!unzip {PATH_DATA+'/processed/far-eastern-fed-district-latest-free.shp.zip'} -d {PATH_DATA+'/processed/far-eastern-fed-district-latest-free.shp'}
!unzip {PATH_DATA+'/processed/north-caucasus-fed-district-latest-free.shp.zip'} -d {PATH_DATA+'/processed/north-caucasus-fed-district-latest-free.shp'}
!unzip {PATH_DATA+'/processed/northwestern-fed-district-latest-free.shp.zip'} -d {PATH_DATA+'/processed/northwestern-fed-district-latest-free.shp'}
!unzip {PATH_DATA+'/processed/siberian-fed-district-latest-free.shp.zip'} -d {PATH_DATA+'/processed/siberian-fed-district-latest-free.shp'}
!unzip {PATH_DATA+'/processed/south-fed-district-latest-free.shp.zip'} -d {PATH_DATA+'/processed/south-fed-district-latest-free.shp'}
!unzip {PATH_DATA+'/processed/ural-fed-district-latest-free.shp.zip'} -d {PATH_DATA+'/processed/ural-fed-district-latest-free.shp'}
!unzip {PATH_DATA+'/processed/volga-fed-district-latest-free.shp.zip'} -d {PATH_DATA+'/processed/volga-fed-district-latest-free.shp'}
!unzip {PATH_DATA+'/processed/kaliningrad-latest-free.shp.zip'} -d {PATH_DATA+'/processed/kaliningrad-latest-free.shp'}

<module 'os' from '/home/dima/anaconda3/envs/ml38/lib/python3.8/os.py'>

In [14]:
folders = [
    'belarus-latest-free.shp',
    'armenia-latest-free.shp',
    'kazakhstan-latest-free.shp',
    'central-fed-district-latest-free.shp',
    'crimean-fed-district-latest-free.shp',
    'far-eastern-fed-district-latest-free.shp',
    'north-caucasus-fed-district-latest-free.shp',
    'northwestern-fed-district-latest-free.shp',
    'siberian-fed-district-latest-free.shp',
    'south-fed-district-latest-free.shp',
    'ural-fed-district-latest-free.shp',
    'volga-fed-district-latest-free.shp',
    'kaliningrad-latest-free.shp',
]

gdf_pois = []
for folder in tqdm(folders):
    path = f'{PATH_DATA}/source/{folder}/gis_osm_pois_free_1.shp'
    gdf_pois.append(gpd.read_file(path))
gdf_pois = pd.concat(gdf_pois, sort=False, ignore_index=True)

  0%|          | 0/13 [00:00<?, ?it/s]

In [19]:
gdf_pois.to_pickle(f'{PATH_DATA}/processed/gdf_pois.pickle')

считаем фичи

In [3]:
gdf_pois = pd.read_pickle(f'{PATH_DATA}/processed/gdf_pois.pickle')

In [4]:
df_voronoi = pd.read_pickle(f'{PATH_DATA}/processed/df_voronoi.pickle')

In [51]:
tags_amenity = [
    'pharmacy',
    'cafe',
    'fast_food',
    'kindergarten',
    'school',
    'restaurant',
    'clinic',
    'dentist',
    'doctors',
    'bar',
    'community_centre',
    'hospital',
    'veterinary',
    'marketplace',
    'college',
    'university',
    'pub',
    'social_facility',
    'arts_centre',
    'theatre',
    'cinema',
    'research_institute',
    'music_school',
    'bbq',
    'public_service',
    'bus_station',
    'public_building' 
]

In [64]:
voronoi_iso_10 = gpd.GeoDataFrame(df_voronoi['id'].copy(), geometry=df_voronoi['voronoi_iso_10'], crs=4326)
voronoi_iso_10 = gdf_pois.query('fclass in @tags_amenity').sjoin(voronoi_iso_10)
voronoi_iso_10 = voronoi_iso_10 \
    .groupby(['id', 'fclass']) \
    .size().unstack('fclass') \
    .fillna(0)
tags_amenity_final = (voronoi_iso_10 > 0) \
    .mean(axis=0) \
    .to_frame('rate') \
    .query('rate > 0.1') \
    .index.tolist()
voronoi_iso_10 = voronoi_iso_10[tags_amenity_final]
voronoi_iso_10.columns = [f'voronoi_iso_10_{col}' for col in voronoi_iso_10.columns]
voronoi_iso_10 = voronoi_iso_10.reset_index()

In [65]:
voronoi_iso_15 = gpd.GeoDataFrame(df_voronoi['id'].copy(), geometry=df_voronoi['voronoi_iso_15'], crs=4326)
voronoi_iso_15 = gdf_pois.query('fclass in @tags_amenity').sjoin(voronoi_iso_15)
voronoi_iso_15 = voronoi_iso_15 \
    .groupby(['id', 'fclass']) \
    .size().unstack('fclass') \
    .fillna(0)
tags_amenity_final = (voronoi_iso_15 > 0) \
    .mean(axis=0) \
    .to_frame('rate') \
    .query('rate > 0.1') \
    .index.tolist()
voronoi_iso_15 = voronoi_iso_15[tags_amenity_final]
voronoi_iso_15.columns = [f'voronoi_iso_15_{col}' for col in voronoi_iso_15.columns]
voronoi_iso_15 = voronoi_iso_15.reset_index()

In [69]:
df_pois = df_voronoi[['id']] \
    .merge(voronoi_iso_10, on='id', how='left') \
    .merge(voronoi_iso_15, on='id', how='left') \
    .fillna(0)

In [70]:
df_pois

,id,voronoi_iso_10_bar,voronoi_iso_10_cafe,voronoi_iso_10_clinic,voronoi_iso_10_dentist,voronoi_iso_10_doctors,voronoi_iso_10_fast_food,voronoi_iso_10_kindergarten,voronoi_iso_10_pharmacy,voronoi_iso_10_restaurant,voronoi_iso_15_bar,voronoi_iso_15_cafe,voronoi_iso_15_clinic,voronoi_iso_15_dentist,voronoi_iso_15_doctors,voronoi_iso_15_fast_food,voronoi_iso_15_kindergarten,voronoi_iso_15_pharmacy,voronoi_iso_15_restaurant,voronoi_iso_15_school
0,18398,0.0,2.0,0.0,0.0,1.0,1.0,0.0,3.0,2.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,3.0,2.0,0.0
1,14532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,23801,0.0,3.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,1.0,3.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0
3,9791,1.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,3.0,0.0,0.0,0.0,2.0,0.0,1.0,3.0,0.0
4,3004,1.0,4.0,0.0,0.0,1.0,1.0,0.0,6.0,0.0,1.0,4.0,0.0,0.0,1.0,1.0,0.0,6.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26012,18085,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26013,23428,1.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,1.0,1.0
26014,11862,1.0,4.0,1.0,1.0,1.0,2.0,0.0,5.0,0.0,1.0,4.0,1.0,1.0,1.0,2.0,0.0,5.0,0.0,0.0
26015,10382,0.0,1.0,1.0,1.0,0.0,2.0,2.0,1.0,2.0,0.0,1.0,1.0,1.0,0.0,3.0,2.0,1.0,3.0,0.0


In [73]:
df_pois.to_pickle(f'{PATH_DATA}/processed/df_pois.pickle')

# Подсчет расстояний

In [11]:
gdf_pois.head()

,osm_id,code,fclass,name,geometry,lon,lat
0,1000,2501,supermarket,Десяточка,POINT (37.20753 55.99511),37.207532,55.995115
1,61669376,2203,cinema,Чайка,POINT (37.46754 56.00921),37.467544,56.009213
2,75724764,2907,camera_surveillance,None,POINT (39.25058 51.82170),39.250579,51.821703
3,90326991,2907,camera_surveillance,None,POINT (41.75275 51.93981),41.752750,51.939811
4,94057611,2721,attraction,Исток Москвы-реки,POINT (35.40442 55.49075),35.404425,55.490753


In [19]:
gdf_pois[gdf_pois['osm_id'].duplicated(keep=False)].sort_values('osm_id')

,osm_id,code,fclass,name,geometry,lon,lat,lon_rad,lat_rad
538841,10001151268,2302,fast_food,#Кебаб,POINT (19.91117 54.65164),19.911168,54.651641,0.347515,0.953851
363622,10001151268,2302,fast_food,#Кебаб,POINT (19.91117 54.65164),19.911168,54.651641,0.347515,0.953851
363623,10001151269,2302,fast_food,Щи-борщи,POINT (19.91136 54.65171),19.911364,54.651709,0.347519,0.953852
538842,10001151269,2302,fast_food,Щи-борщи,POINT (19.91136 54.65171),19.911364,54.651709,0.347519,0.953852
363624,10001151270,2517,shoe_shop,Respect,POINT (19.91154 54.65176),19.911542,54.651762,0.347522,0.953853
...,...,...,...,...,...,...,...,...,...
963812,9996853486,2523,stationery,Карандаш,POINT (58.68279 52.71843),58.682786,52.718432,1.024208,0.920110
538743,9998119504,2742,viewpoint,None,POINT (59.76411 64.97775),59.764105,64.977747,1.043080,1.134076
963817,9998119504,2742,viewpoint,None,POINT (59.76411 64.97775),59.764105,64.977747,1.043080,1.134076
538744,9998119508,2742,viewpoint,None,POINT (59.76478 64.97796),59.764783,64.977958,1.043092,1.134079


In [22]:
gdf_pois['osm_id'].duplicated().sum(), gdf_pois.shape

(14548, (1115149, 9))

In [23]:
gdf_pois = gdf_pois.drop_duplicates(subset=['osm_id'])

In [10]:
gdf_pois['lon'] = gdf_pois['geometry'].apply(lambda p: p.x)
gdf_pois['lat'] = gdf_pois['geometry'].apply(lambda p: p.y)

gdf_pois['lon_rad'] = gdf_pois['lon'].apply(np.deg2rad).values
gdf_pois['lat_rad'] = gdf_pois['lat'].apply(np.deg2rad).values

In [24]:
gdf_pois

,osm_id,code,fclass,name,geometry,lon,lat,lon_rad,lat_rad
0,1000,2501,supermarket,Десяточка,POINT (37.20753 55.99511),37.207532,55.995115,0.649394,0.977299
1,61669376,2203,cinema,Чайка,POINT (37.46754 56.00921),37.467544,56.009213,0.653932,0.977545
2,75724764,2907,camera_surveillance,None,POINT (39.25058 51.82170),39.250579,51.821703,0.685052,0.904459
3,90326991,2907,camera_surveillance,None,POINT (41.75275 51.93981),41.752750,51.939811,0.728723,0.906521
4,94057611,2721,attraction,Исток Москвы-реки,POINT (35.40442 55.49075),35.404425,55.490753,0.617924,0.968496
...,...,...,...,...,...,...,...,...,...
1115144,11403451816,2724,memorial,Послание налоговикам Оренбуржья 2040 года,POINT (55.12176 51.79914),55.121758,51.799140,0.962056,0.904066
1115145,11403451817,2724,memorial,Закладной камень аллеи «25-летие Федеральной н...,POINT (55.12176 51.79914),55.121760,51.799140,0.962056,0.904066
1115146,11403898260,2302,fast_food,СушиШок,POINT (49.09165 55.86144),49.091649,55.861440,0.856811,0.974966
1115147,11403915602,2111,clinic,Айболит,POINT (49.09754 55.85995),49.097540,55.859951,0.856914,0.974940


In [26]:
from sklearn.neighbors import BallTree

In [27]:
ball = BallTree(gdf_pois[["lat_rad", "lon_rad"]].values, metric='haversine')
dic_index2id = dict(enumerate(gdf_pois['osm_id'].values))

In [40]:
ind, dist = ball.query_radius(X=df_voronoi[["lat_rad", "lon_rad"]].values, r=3000/6371000, count_only=False, return_distance=True)

In [93]:
%%time

dists = pd.Series(ind)
dists.index = df_voronoi['id']
dists = dists.explode().to_frame('osm_id')
dists['osm_id'] = dists['osm_id'].map(dic_index2id)
dists['dist'] = pd.Series(dist).explode().values * 6371000
dists = dists.reset_index()
dists = dists.merge(gdf_pois[['osm_id', 'fclass']], on='osm_id')
mask = dists.groupby(['id', 'fclass'])['dist'].rank(method='first', ascending=False) == 1
dists = dists[mask]
dists = dists.set_index(['id', 'fclass'])['dist'].unstack('fclass').fillna(999999)
dists.columns = [f'{col}_dist' for col in dists.columns]
dists = dists.reset_index()
dists = df_voronoi[['id']].merge(dists, on='id', how='left').fillna(999999)

CPU times: user 2min 37s, sys: 4.15 s, total: 2min 41s
Wall time: 2min 40s


In [100]:
dists

,id,alpine_hut_dist,archaeological_dist,arts_centre_dist,artwork_dist,atm_dist,attraction_dist,bakery_dist,bank_dist,bar_dist,battlefield_dist,beauty_shop_dist,bench_dist,beverages_dist,bicycle_rental_dist,bicycle_shop_dist,biergarten_dist,bookshop_dist,butcher_dist,cafe_dist,camera_surveillance_dist,camp_site_dist,car_dealership_dist,car_rental_dist,car_sharing_dist,car_wash_dist,caravan_site_dist,castle_dist,chalet_dist,chemist_dist,cinema_dist,clinic_dist,clothes_dist,college_dist,comms_tower_dist,community_centre_dist,computer_shop_dist,convenience_dist,courthouse_dist,dentist_dist,department_store_dist,doctors_dist,dog_park_dist,doityourself_dist,drinking_water_dist,embassy_dist,fast_food_dist,fire_station_dist,florist_dist,food_court_dist,fort_dist,fountain_dist,furniture_shop_dist,garden_centre_dist,general_dist,gift_shop_dist,golf_course_dist,graveyard_dist,greengrocer_dist,guesthouse_dist,hairdresser_dist,hospital_dist,hostel_dist,hotel_dist,hunting_stand_dist,ice_rink_dist,jeweller_dist,kindergarten_dist,kiosk_dist,laundry_dist,library_dist,lighthouse_dist,mall_dist,market_place_dist,memorial_dist,mobile_phone_shop_dist,monument_dist,motel_dist,museum_dist,newsagent_dist,nightclub_dist,nursing_home_dist,observation_tower_dist,optician_dist,outdoor_shop_dist,park_dist,pharmacy_dist,picnic_site_dist,pitch_dist,playground_dist,police_dist,post_box_dist,post_office_dist,prison_dist,pub_dist,public_building_dist,recycling_dist,recycling_clothes_dist,recycling_glass_dist,recycling_metal_dist,recycling_paper_dist,restaurant_dist,ruins_dist,school_dist,shelter_dist,shoe_shop_dist,sports_centre_dist,sports_shop_dist,stadium_dist,stationery_dist,supermarket_dist,swimming_pool_dist,telephone_dist,theatre_dist,theme_park_dist,toilet_dist,tourist_info_dist,tower_dist,town_hall_dist,toy_shop_dist,track_dist,travel_agent_dist,university_dist,vending_any_dist,vending_cigarette_dist,vending_machine_dist,vending_parking_dist,veterinary_dist,video_shop_dist,viewpoint_dist,waste_basket_dist,wastewater_plant_dist,water_mill_dist,water_tower_dist,water_well_dist,water_works_dist,wayside_cross_dist,wayside_shrine_dist,windmill_dist,zoo_dist
0,12017,999999.0,999999.0,1129.690331,2784.373607,2954.600343,2046.723057,2940.262584,2984.262564,2915.094309,999999.0,2986.786082,2958.448926,2570.189666,1165.637417,2911.662379,999999.000000,2997.101884,2572.973381,2999.041281,2848.188399,953.943095,2727.559899,1314.652212,999999.0,2968.737592,999999.000000,999999.000000,1082.256137,2521.725814,2078.150835,2999.870052,2919.344064,2915.974721,2866.413924,2264.556649,2996.170984,2989.187298,1463.643846,2943.431850,2825.770382,2871.362550,999999.0,2935.099955,2996.409054,2747.978704,2992.746115,2001.680350,2976.676666,2311.299465,999999.0,2701.773281,2995.788889,1793.50427,999999.000000,2879.428481,999999.0,999999.0,2961.369872,2135.563358,2986.881373,2938.668758,2956.010137,2885.851678,999999.0,999999.0,2337.711030,2880.096838,2920.791748,2015.793234,2208.855257,999999.0,2157.054438,2229.396442,2990.933459,2844.414959,2092.761226,2851.710947,2512.693560,2995.716372,2776.936486,999999.0,999999.0,2780.073440,2967.588041,999999.00000,2974.380029,999999.000000,2370.470331,2999.424310,2793.378187,1446.263227,2945.540983,999999.0,1537.191273,999999.0,2808.326696,999999.0,2919.869800,999999.0,2921.120293,2990.550996,885.775539,2982.942269,2989.212307,2822.482123,1594.975878,2673.446722,999999.000000,2943.613400,2975.302875,999999.000000,2330.274544,1449.785685,999999.000000,2537.521940,1602.779930,2723.675175,973.496704,2730.575655,999999.0,2622.676067,2918.529129,2749.541512,999999.0,999999.0000,999999.0,2927.195092,999999.0,2875.565731,2687.800089,999999.0,999999.0,999999.000000,999999.000000,999999.0,999999.0,999999.0,999999.0,1476.736855
1,752,999999.0,999999.0,999999.000000,999999.000000,443.108170,999999.000000,999999.000000,444.769473,999999.000000,999999.0,999999.000000,999999.000000,1917.539324,999999.000000,999999.000000,999999.000000,999999.000000,3

In [101]:
dists.to_pickle(f'{PATH_DATA}/processed/df_pois_dists.pickle')